In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
#using model.save_weights() to save the weights of the model in HDF5 format
# model.save_weights("/content/gdrive/My Drive/weights.h5")

Mounted at /content/gdrive


In [50]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer

from huggingface_hub import login

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install accelerate

import accelerate

In [51]:
model_name = "Salesforce/codegen-350M-mono"
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
device_map = {"": 0}

In [52]:
peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)

In [53]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

In [54]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

In [64]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [63]:
code_dataset = load_dataset(dataset_name, split='train[:5000]')



model = AutoModelForCausalLM.from_pretrained(model_name,
          quantization_config=bnb_config,
          use_cache = False,
          device_map=device_map)
model.config.pretraining_tp = 1


In [59]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:

tokenized_code_dataset = code_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=code_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2122 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5271 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4658 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (11388 > 2048). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (6735 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4305 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5956 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (12474 > 2048). Running this sequence through the model will result in indexing errors


In [14]:

block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [15]:
lm_code_dataset = tokenized_code_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
lm_code_dataset['train'][0]

In [16]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [65]:
trainingArgs = TrainingArguments(
    output_dir="./colab_code_model_UT",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    seed=42
)



trainer = SFTTrainer(
    model=model,
    train_dataset=code_dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (2905 > 2048). Running this sequence through the model will result in indexing errors
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.167, 'learning_rate': 1.7543859649122806e-05, 'epoch': 0.01}
{'loss': 1.1787, 'learning_rate': 3.508771929824561e-05, 'epoch': 0.02}
{'loss': 1.1399, 'learning_rate': 5.2631578947368424e-05, 'epoch': 0.02}
{'loss': 1.0771, 'learning_rate': 7.017543859649122e-05, 'epoch': 0.03}
{'loss': 1.03, 'learning_rate': 8.771929824561403e-05, 'epoch': 0.04}
{'loss': 0.9896, 'learning_rate': 0.00010526315789473685, 'epoch': 0.05}
{'loss': 0.9967, 'learning_rate': 0.00012280701754385965, 'epoch': 0.06}
{'loss': 0.9238, 'learning_rate': 0.00014035087719298245, 'epoch': 0.06}
{'loss': 0.8468, 'learning_rate': 0.00015789473684210527, 'epoch': 0.07}
{'loss': 0.8166, 'learning_rate': 0.00017543859649122806, 'epoch': 0.08}
{'loss': 0.8306, 'learning_rate': 0.00019298245614035088, 'epoch': 0.09}
{'loss': 0.8193, 'learning_rate': 0.00019999865623437013, 'epoch': 0.1}
{'loss': 0.8693, 'learning_rate': 0.000199990444464082, 'epoch': 0.1}
{'loss': 0.7694, 'learning_rate': 0.00019997476807225985, 'ep

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.8092, 'learning_rate': 0.00019995162822919883, 'epoch': 1.0}
{'loss': 0.7422, 'learning_rate': 0.00019992102666236566, 'epoch': 1.01}
{'loss': 0.6731, 'learning_rate': 0.00019988296565626987, 'epoch': 1.02}
{'loss': 0.7944, 'learning_rate': 0.00019983744805229296, 'epoch': 1.03}
{'loss': 0.7189, 'learning_rate': 0.00019978447724847652, 'epoch': 1.03}
{'loss': 0.6953, 'learning_rate': 0.0001997240571992685, 'epoch': 1.04}
{'loss': 0.7354, 'learning_rate': 0.0001996561924152278, 'epoch': 1.05}
{'loss': 0.6852, 'learning_rate': 0.00019958088796268793, 'epoch': 1.06}
{'loss': 0.7519, 'learning_rate': 0.00019949814946337838, 'epoch': 1.07}
{'loss': 0.6488, 'learning_rate': 0.00019940798309400526, 'epoch': 1.07}
{'loss': 0.6707, 'learning_rate': 0.00019931039558578997, 'epoch': 1.08}
{'loss': 0.719, 'learning_rate': 0.0001992053942239668, 'epoch': 1.09}
{'loss': 0.7186, 'learning_rate': 0.00019909298684723904, 'epoch': 1.1}
{'loss': 0.7851, 'learning_rate': 0.00019897318184719385,

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.6765, 'learning_rate': 0.00019871141530411853, 'epoch': 2.0}
{'loss': 0.67, 'learning_rate': 0.00019856947330283752, 'epoch': 2.01}
{'loss': 0.6736, 'learning_rate': 0.00019842017276027832, 'epoch': 2.02}
{'loss': 0.6491, 'learning_rate': 0.00019826352482222638, 'epoch': 2.03}
{'loss': 0.6842, 'learning_rate': 0.0001980995411829749, 'epoch': 2.04}
{'loss': 0.6573, 'learning_rate': 0.00019792823408445174, 'epoch': 2.04}
{'loss': 0.8004, 'learning_rate': 0.00019774961631530545, 'epoch': 2.05}
{'loss': 0.6713, 'learning_rate': 0.00019756370120995066, 'epoch': 2.06}
{'loss': 0.7064, 'learning_rate': 0.0001973705026475726, 'epoch': 2.07}
{'loss': 0.6803, 'learning_rate': 0.00019717003505109095, 'epoch': 2.08}
{'loss': 0.6906, 'learning_rate': 0.00019696231338608316, 'epoch': 2.08}
{'loss': 0.6687, 'learning_rate': 0.0001967473531596671, 'epoch': 2.09}
{'loss': 0.6842, 'learning_rate': 0.00019652517041934356, 'epoch': 2.1}
{'loss': 0.6318, 'learning_rate': 0.0001962957817517982, '

TrainOutput(global_step=222, training_loss=0.7840495141776832, metrics={'train_runtime': 3970.6604, 'train_samples_per_second': 3.778, 'train_steps_per_second': 0.472, 'train_loss': 0.7840495141776832, 'epoch': 2.12})

In [80]:
trainer.save_model('./saved')

In [81]:
trained_model = AutoModelForCausalLM.from_pretrained('./saved')

In [83]:
# merged = trained_model.merge_and_unload()
# merged.save_pretrained("merged",safe_serialization=True)
# tokenizer.save_pretrained("merged")

#push merged model to the hub
trained_model.push_to_hub("01GangaPutraBheeshma/colab_code_model_UT")
tokenizer.push_to_hub("01GangaPutraBheeshma/colab_code_model_UT")

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/01GangaPutraBheeshma/colab_code_model_UT/commit/0bae9edcf021299ca46d5ca78183a57f244c2d49', commit_message='Upload tokenizer', commit_description='', oid='0bae9edcf021299ca46d5ca78183a57f244c2d49', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# import math

# eval_results = trainer.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [74]:
instruction="Write a Python program to generate a Markov chain given a text input."
input="Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'"

prompt = f"""### Instruction:
Write a Python program to generate a Markov chain given a text input.
"""


# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("01GangaPutraBheeshma/colab_code_model_UT")
# model = AutoModelForCausalLM.from_pretrained("01GangaPutraBheeshma/colab_code_model_UT")


tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer(prompt, return_tensors="pt").input_ids

trained_model = AutoModelForCausalLM.from_pretrained("01GangaPutraBheeshma/colab_code_model_UT")
outputs_1 = trained_model.generate(inputs, max_new_tokens=128, do_sample=True, top_k=50, top_p=0.95)

# outputs_2 = trained_model.generate(**inputs, num_beams=4, do_sample=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [75]:

print(f"-------------------------\n\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs_1.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

-------------------------


Generated instruction:
 
 # # #   I n p u t : 
 " T h i s   i s   a   s e n t e n c e. " 
 
 # # #   O u t p u t : 
 i m p o r t   r a n d o m   
     
 d e f   g e n e r a t e _ h a n d l e r ( t e x t ) :   
         h a n d l e r   =   " T h i s   i s   a   s e n t e n c e
-------------------------




In [84]:
instruction="Write a Python program to generate a Markov chain given a text input."
input="Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'"

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
# with torch.inference_mode():

print(f"-------------------------\n\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n\n")

print(f"Before Training Response :")
output_before = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)
print(f"Generated instruction:\n{tokenizer.batch_decode(output_before.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

print(f"After Training Response :")
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

print(f"-------------------------\n\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------------------------


Prompt:
### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
Write a Python program to generate a Markov chain given a text input.

### Input:
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'

### Response:


-------------------------


Before Training Response :


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkp

Generated instruction:
#!/usr/bin/python

import random

def markov_chain(text):
    """Generates a Markov chain given a text input.

    Args:
        text (str): The text input.

    Returns:
        str: The Markov chain.
    """
    # initialize an empty list to store the Markov chains
    chains = []

    # loop through each word in the text
    for word in text.
-------------------------


After Training Response :
-------------------------


Generated instruction:
import random

def markov_chain(text):
    text = text.split()
    chain = []
    for i in range(len(text)):
        word = text[i]
        if i == 0:
            chain.append(word)
        else:
            chain.append(chain[-1] + " " + word)
    return chain

text = "Alice was beginning to get very tired of sitting by
-------------------------




In [85]:
instruction="Write a Python program to generate a binary search  given an array input and element to be searched."
input="[2,3,4,5], [5]"

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
# with torch.inference_mode():

print(f"-------------------------\n\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n\n")

print(f"Before Training Response :")
output_before = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)
print(f"Generated instruction:\n{tokenizer.batch_decode(output_before.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

print(f"After Training Response :")
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

print(f"-------------------------\n\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------------------------


Prompt:
### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
Write a Python program to generate a binary search  given an array input and element to be searched.

### Input:
[2,3,4,5], [5]

### Response:


-------------------------


Before Training Response :


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated instruction:
def binary_search(arr, target):
    left = 0
    right = len(arr) - 1

    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] > target:
            right = mid - 1
        else:
            left = mid + 1
    return -1

arr = [2,3,4
-------------------------


After Training Response :
-------------------------


Generated instruction:
def binary_search(arr, elem):
    low = 0
    high = len(arr) - 1
    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == elem:
            return mid
        elif arr[mid] > elem:
            high = mid - 1
        else:
            low = mid + 1
    
-------------------------


